In [119]:
from shutil import copyfile
import os.path

def get_packetlog_columns():
    return [ 'extended_seq',
             'tracked_seq',
             'state',
             'tracked_ntp',
             'marker',
             'header_size',
             'payload_size',
             'timestamp',
             'payload_type',
             'played_out'];

def get_delta_packetlog_columns():
    return ["owd",
            "BiF",
            "playout_delay"];

def get_statslog_columns():
    return [ 'rcved_tot_packets',
             'rcved_tot_bytes',
             'rcved_acc_bytes',
             'rcved_acc_packets',                             

             'lost_tot_packets',
             'lost_tot_bytes',
             'lost_acc_bytes',
             'lost_acc_packets',

             'discarded_tot_packets',
             'discarded_tot_bytes',
             'discarded_acc_bytes',
             'discarded_acc_packets',

             'corrupted_tot_packets',
             'corrupted_tot_bytes',
             'corrupted_acc_bytes',
             'corrupted_acc_packets',

             'repaired_tot_packets',
             'repaired_tot_bytes',
             'repaired_acc_bytes',
             'repaired_acc_packets',

             'fec_tot_packets',
             'fec_tot_bytes',
             'fec_acc_bytes',
             'fec_acc_packets',
            ];
def get_veth_columns():
    return ["tcp_bytes",
            "tcp_packets",
            "tcp_flowsnum",
            "bw_forward", 
            "bw_backward"];

def get_bytefactor():

    #bytes with only rtp:
    return 20

    #bytes with mprtp and abs time header ext
    #return 28

In [2]:
from shutil import copyfile
import os.path
import pandas as pd
import datetime

#Make owd_statlogs
def make_merged_packetlogs(snd_packetlogs, rcv_packetlogs, merged_packetlogs_path):
    packetlog_columns     = get_packetlog_columns()
    snd_packets = pd.read_csv(snd_packetlogs, names=packetlog_columns)
    rcv_packets = pd.read_csv(rcv_packetlogs, names=packetlog_columns)
    
    result = pd.merge(snd_packets, rcv_packets, on='extended_seq', \
                           suffixes=['_snd','_rcv'])

    result.to_csv(merged_packetlogs_path, sep=',')
    
    #Make owd_statlogs
def make_path_statlogs(veth_logfile, 
#                        targets_logfile,
                       snd_statlogfile, 
                       rcv_statlogfile,
                       delta_packetlogfile, 
                       result_filepath):
    
    bytefactor          = get_bytefactor()
    statlog_columns     = get_statslog_columns()
    
    delta_columns = get_delta_packetlog_columns()
   
    #targets_stats = pd.read_csv(targets_logfile,     names=["target"])
    veth_stats    = pd.read_csv(veth_logfile,        names=get_veth_columns())
    snd_stats     = pd.read_csv(snd_statlogfile,     names=statlog_columns)
    rcv_stats     = pd.read_csv(rcv_statlogfile,     names=statlog_columns)
    packet_dstats = pd.read_csv(delta_packetlogfile, names=delta_columns)
    
    sending_rates = snd_stats.apply(lambda record: record['rcved_acc_packets'] * bytefactor + 
                                    record['rcved_acc_bytes'], axis=1)
    # sending_rates = snd_stats.apply(lambda record: record['rcved_acc_bytes'], axis=1)
    
    fec_rates     = snd_stats.apply(lambda record: record['fec_acc_packets'] * bytefactor + 
                                    record['fec_acc_bytes'], axis=1)
    
    ffre          = rcv_stats.apply(lambda record:  
                                    record['repaired_acc_packets'] / (record['repaired_acc_packets'] + record['lost_acc_packets']) if 0 < record['repaired_acc_packets'] else 0, axis=1)
    
    #print(rcv_stats['lost_acc_packets'])
    #print(rcv_stats['repaired_acc_packets'])
    #print(packet_dstats);
    result = pd.concat([veth_stats["bw_forward"], 
                        veth_stats["bw_backward"], 
                        sending_rates,
                        fec_rates,
                        packet_dstats['owd'],
                        packet_dstats['BiF'],
                        ffre,
#                         targets_stats["target"],
                        packet_dstats['playout_delay'],
                        rcv_stats["lost_acc_packets"],
                        rcv_stats["repaired_acc_packets"],
                        veth_stats["tcp_flowsnum"],
                        veth_stats["tcp_bytes"],
                        veth_stats["tcp_packets"],                        
                       ], 
                       axis=1)
#    print(result)
    result.to_csv(result_filepath, sep=',')



In [120]:
from shutil import copyfile
import os.path
import pandas as pd
import datetime

def evaluate_statslog(statslog_file, 
                      packetlogs_file, 
                      veth_logfile, 
                      dpck_logfile,
                      sampling_num):
    
    bytefactor        = get_bytefactor() 
    packetlog_columns = get_packetlog_columns()
    statlog_table     = pd.read_csv(statslog_file, names=get_statslog_columns())
    veth_table        = pd.read_csv(veth_logfile,  names=["tcps","bw"])
    dpcklog_table     = pd.read_csv(dpck_logfile,  names=get_delta_packetlog_columns()).head(sampling_num + 1).tail(sampling_num)
    goodput_helper    = pd.Series(((statlog_table["rcved_acc_bytes"] - statlog_table["discarded_acc_bytes"]) + 
                                 (statlog_table["rcved_acc_packets"] - statlog_table["discarded_acc_packets"]) * bytefactor ) / 125)
    
    owdmean_helper    = pd.Series(dpcklog_table["owd"] / 1000)

    #shaping
    available_sampling_num = len(statlog_table["rcved_acc_packets"])
    if(available_sampling_num < sampling_num) : 
        print("Required sampling number is " + str(sampling_num) + 
              " available: " + str(available_sampling_num) + " at " + statslog_file)
        return
    
    print("Required sampling num is " + str(sampling_num) + " : " + str(len(statlog_table["rcved_acc_packets"])))
    
    statlog_table     = statlog_table.head(sampling_num + 1).tail(sampling_num)
    veth_table        = veth_table.head(sampling_num + 1).tail(sampling_num)
    dpcklog_table     = dpcklog_table.head(sampling_num + 1).tail(sampling_num)
    goodput_helper    = goodput_helper.head(sampling_num + 1).tail(sampling_num)

    index = sampling_num - 1
    rcved_packets     = statlog_table.iloc[index]["rcved_tot_packets"]
    repaired_packets  = statlog_table.iloc[index]["repaired_tot_packets"]
    rcved_packets     = statlog_table.iloc[index]["rcved_tot_packets"]
    rcved_bytes       = statlog_table.iloc[index]["rcved_tot_bytes"]
    discarded_packets = statlog_table.iloc[index]["discarded_tot_packets"]
    discarded_bytes   = statlog_table.iloc[index]["discarded_tot_bytes"]
    lost_packets      = statlog_table.iloc[index]["lost_tot_packets"]
    received_packets  = statlog_table.iloc[index]["rcved_tot_packets"]
    lost_rate         = float(lost_packets) / float(received_packets + lost_packets)
    packetlogs_table  = pd.read_csv(packetlogs_file, names=packetlog_columns).head(received_packets + 1).tail(received_packets)
    

    frame_is_lost   = False
    lost_frames     = 0
    received_frames = 0    
    last_timestamp  = 0;
    
    for index, row in packetlogs_table.iterrows():
        if(row["timestamp"] != 0):
            last_timestamp = row["timestamp"]
            break

    for index, row in packetlogs_table.iterrows():
        if(row["timestamp"] == 0 or row["state"] != 1):
            frame_is_lost = True;
            continue;
        if(row["timestamp"] != last_timestamp):
            last_timestamp = row["timestamp"]
            if(frame_is_lost):
                lost_frames += 1
            else: 
                received_frames += 1
            frame_is_lost = False;
    repaired_rate = 0.
    if(0 < lost_packets):
        repaired_rate = float(repaired_packets) / float(repaired_packets + lost_packets)
    
    lost_frame_rate = 0;
    if(0 < received_frames):
        lost_frame_rate = float(lost_frames) / float(lost_frames + received_frames)
    
    return{
        'repaired_packets_num':    repaired_packets,
        'repaired_rate':           repaired_rate,
        'lost_packet_rate':        lost_rate, 
        'goodput_mean':            goodput_helper.mean(),
        'goodput_std':             goodput_helper.std(),
        'lost_packets_num':        lost_packets,
        'rcved_packets_num':       rcved_packets,
        'lost_frames_num':         lost_frames,
        'lost_frame_rate':         lost_frame_rate,
        'received_frames':         received_frames,
        'owd mean':                owdmean_helper.mean(),
        'owd std':                 owdmean_helper.std(),
        'received_bytes':          rcved_bytes,
    };

def evaluate_videofiles(original, processed, width, height, framesnum, targetname):
    psnr_file   = targetname + "_psnr.csv"
    ssim_file   = targetname + "_ssim.csv"
    msssim_file = targetname + "_msssim.csv"
    vifp_file   = targetname + "_vifp.csv"
    columns     = ["frame","value"]
    
    print("Evaluating Video")
    print("Source: " + original +" Processed: " + processed + " W: " + str(width) + " H: " + str(height) + " psnr: " + psnr_file)
    print("Framesnum: " + str(framesnum))
    
    #!vqmt $original $processed $height $width $framesnum 1 $targetname PSNR SSIM MSSSIM VIFP
    !vqmt $original $processed $height $width $framesnum 1 $targetname PSNR 
    
    psnr    = pd.Series(pd.read_csv(psnr_file,    names=columns).head(-1).tail(-1).astype('float')["value"])
    #ssim    = pd.Series(pd.read_csv(ssim_file,    names=columns).head(-1).tail(-1).astype('float')["value"])
    #msssim  = pd.Series(pd.read_csv(msssim_file,  names=columns).head(-1).tail(-1).astype('float')["value"])
    #vifp    = pd.Series(pd.read_csv(vifp_file,    names=columns).head(-1).tail(-1).astype('float')["value"])
    
    return {
        "psnr_mean": psnr.mean(),
        "psnr_std":  psnr.std(),
        #"ssim_mean": ssim.mean(),
        #"ssim_std":  ssim.std(),
        #"msssim_mean": msssim.mean(),
        #"msssim_std":  msssim.std(),
        #"vifp_mean": vifp.mean(),
        #"vifp_std":  vifp.std(),
        };
    
    

In [121]:
#!cd ../../temp; ls #The directory where the logfiles are
base_path       = "../../"
plot_path       = "../gnuplots"
temp_path       = base_path + "/temp" 


In [123]:
#Save measurement results
import csv

test      = "rmcat1"
algorithm = "scream"
# algorithm = "scream"
count     = "1"
owd       = "300ms" 
# owd      = "100ms" 
# owd      = "300ms" 
jitter    = "0ms"
target    = "_".join([algorithm, count, owd, jitter])

evaluate_videofiles(base_path + "produced.yuv", 
                    base_path + "consumed.yuv", 
                    352,  #width
                    288,  #height
                    2000, #frames
                    temp_path + "/vqmt")

save_destination = base_path + "scripts/saves/" + test + "/" + target
if(os.path.exists(save_destination) == False): 
    !mkdir $save_destination
!cp $temp_path/* $save_destination


Evaluating Video
Source: ../..//produced.yuv Processed: ../..//consumed.yuv W: 352 H: 288 psnr: ../..//temp/vqmt_psnr.csv
Framesnum: 2000
Time: 109.789s


In [118]:
import os
import csv

# - - - - - - RMCA4 Config - - - - - - - - - -
test              = "rmcat4"
test_path         = base_path + "scripts/saves/" + test + "/"
touched           = {}
flownum           = 1

flowstat_suffixes = ["","2","3"]
flowcsv_prefixes  = ["flow1", "flow2", "flow3"]

viewpdf           = base_path + "/temp/statlogs.pdf"
 


for dirname, dirnames, filenames in os.walk(test_path):
    for targetdir in dirnames:
        algorithm, count, owd, jitter = targetdir.split('_')
        meas_path = base_path + "scripts/saves/rmcat4/" + '_'.join([algorithm, count, owd, jitter])
        print("--- Operate in " + meas_path + " ---")
        
        bandwidth        = meas_path + "/bandwidth.csv"
        veth0_stats      = meas_path + "/veth0_stats.csv"
        rmcat_tmplogs    = meas_path + "/rmcat_tmplogs.csv"
        plotfile         = plot_path + "/" + test + ".plot"
        outplot          = meas_path + "/" + '_'.join([algorithm,owd,jitter]) + ".pdf"
        gnuplot_param    = ""
        
        for flowcounter in range(0, 3):
            
            stat_suffix      = flowstat_suffixes[flowcounter]
            
            snd_statlogs     = meas_path + "/snd_statlogs"    + stat_suffix + ".csv"
            rcv_statlogs     = meas_path + "/rcv_statlogs"    + stat_suffix + ".csv"
            snd_packetlogs   = meas_path + "/snd_packetlogs"  + stat_suffix + ".csv"
            rcv_packetlogs   = meas_path + "/rcv_packetlogs"  + stat_suffix + ".csv"
            merged_statlogs  = meas_path + "/merged_statlogs" + stat_suffix + ".csv"  
            delta_statlogs   = meas_path + "/delta_statlogs"  + stat_suffix + ".csv"
            rmcat_statlogs   = meas_path + "/rmcat_statlogs"  + stat_suffix + ".csv"
            evalfile         = meas_path + "/evaluation"      + stat_suffix + ".csv"

            gnuplot_param   += "-e statlogs='" + rmcat_statlogs + "' "
            
            #Made merged statlogs
            make_merged_packetlogs(snd_packetlogs, rcv_packetlogs, merged_statlogs)
            !./../../make_delta_statlogs $merged_statlogs $delta_statlogs

            make_path_statlogs(veth0_stats, 
            #              targetsfile, 
                           snd_statlogs, 
                           rcv_statlogs, 
                           delta_statlogs, 
                           rmcat_statlogs)
            
            !sed '1d' $rmcat_statlogs > $rmcat_tmplogs; mv $rmcat_tmplogs $rmcat_statlogs
            
            !gnuplot $gnuplot_params \
                 -e "output_file='$outplot'" \
                 $plotfile
            
            flowcsv_prefix = flowcsv_prefixes[flowcounter]
            
            eval_result = evaluate_statslog(rcv_statlogs, rcv_packetlogs, veth0_stats, delta_statlogs, 1000)
     
            resultfile = path + '_'.join([flowcsv_prefix, algorithm, owd, jitter]) +".csv"
            csv_append(touched, resultfile, eval_result)


--- Operate in ../../scripts/saves/rmcat4/fractal_1_50ms_0ms ---
Make Delta Packet Statlogs.
Source file:             ../../scripts/saves/rmcat4/fractal_1_50ms_0ms/merged_statlogs.csv
Destination file:        ../../scripts/saves/rmcat4/fractal_1_50ms_0ms/delta_statlogs.csv
Accumulation length:     1000ms
Sampling time:           100ms
Unit of delay:           us
Unit of bytes in flight: byte
Elapsed time:            0.165000s
Nr. of processed rows:   14762
Nr. of recorded sample:  1754
Delta PacketStatlog is made
Required sampling num is 1000 : 1782
../../scripts/saves/rmcat1/flow1_fractal_50ms_0ms.csv is removed
------ Obj to save to ../../scripts/saves/rmcat1/flow1_fractal_50ms_0ms.csv------
Make Delta Packet Statlogs.
Source file:             ../../scripts/saves/rmcat4/fractal_1_50ms_0ms/merged_statlogs2.csv
Destination file:        ../../scripts/saves/rmcat4/fractal_1_50ms_0ms/delta_statlogs2.csv
Accumulation length:     1000ms
Sampling time:           100ms
Unit of delay:         